In [1]:
import numpy as np
import cv2
import serial

In [4]:
#Setting up your arduino
arduino = serial.Serial('COM4',9600)

In [3]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

In [4]:
with_mask.shape

(200, 50, 50, 3)

In [5]:
without_mask.shape

(200, 50, 50, 3)

In [6]:
# converting 4D data to 2D data
with_mask = with_mask.reshape(200, 50*50*3)
without_mask = without_mask.reshape(200, 50*50*3)

In [7]:
with_mask.shape

(200, 7500)

In [8]:
without_mask.shape

(200, 7500)

In [9]:
# concatinate the rows
x = np.r_[without_mask, with_mask]

In [10]:
x.shape

(400, 7500)

In [11]:
# creating np array  of 400 zeros
labels = np.zeros(x.shape[0])

In [12]:
# first 200 pointing to 0 (without mask) and next 200 pointing to 1 (withmask)
labels[200:] = 1.0

In [13]:
names = {1 : 'ACCESS GRANTED', 0 : 'ACCESS DENIED'}

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size=0.40, random_state=42)

In [16]:
y_train.shape

(240,)

In [17]:
x_train.shape

(240, 7500)

In [18]:
# PCA - Principal Component Analysis
from sklearn.decomposition import PCA

In [19]:
# Dimensionality reduction using PCA
pca = PCA(n_components=3)
x_train = pca.fit_transform(x_train)

In [20]:
x_train[0]

array([1439.3275806 , -432.38595946,   20.0068858 ])

In [21]:
x_train.shape

(240, 3)

In [22]:
from sklearn import svm

In [23]:
clf = svm.SVC()

In [24]:
clf.fit(x_train, y_train)

SVC()

In [25]:
x_test = pca.transform(x_test)
y_pred = clf.predict(x_test)

In [26]:
haar_data = cv2.CascadeClassifier('data.xml')
capture = cv2.VideoCapture(0)
data = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50,50))
            face = face.reshape(1, -1)
            face = pca.transform(face)
            pred = clf.predict(face)[0]
            n = names[int(pred)]
            if n=='ACCESS GRANTED' :
                cv2.rectangle(img, (x,y), (x+y, y+w), (0,255,0), 4)
                cv2.putText(img, n, (x,y), font, 1, (0,250,0), 2)
                print(n)
                arduino.write(b'1')
            else :
                cv2.rectangle(img, (x,y), (x+y, y+w), (0,0,255), 4)
                cv2.putText(img, n, (x,y), font, 1, (0,0,250), 2)
                print(n)
                arduino.write(b'0')
                
        cv2.imshow('Result', img)
        if cv2.waitKey(2)==27:
            break;
capture.release()
cv2.destroyAllWindows()

ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS DENIED
ACCESS

SerialException: WriteFile failed (PermissionError(13, 'Access is denied.', None, 5))